In [1]:
import sys

if 'google.colab' in sys.modules:
    ! [ -e /content ] && pip install -Uqq fastbook
    !pip install nbdev
    
import fastbook
fastbook.setup_book()

In [2]:
from fastai.vision.all import *
from fastbook import *

# Data

In [5]:
# Download full MNIST dataset
path = untar_data(URLs.MNIST)
path

Path('/Users/adgb/.fastai/data/mnist_png')

In [4]:
# Review directory
path.ls()

(#2) [Path('/Users/adgb/.fastai/data/mnist_png/training'),Path('/Users/adgb/.fastai/data/mnist_png/testing')]

In [6]:
# Review training directory
(path/'training').ls()

(#10) [Path('/Users/adgb/.fastai/data/mnist_png/training/9'),Path('/Users/adgb/.fastai/data/mnist_png/training/0'),Path('/Users/adgb/.fastai/data/mnist_png/training/7'),Path('/Users/adgb/.fastai/data/mnist_png/training/6'),Path('/Users/adgb/.fastai/data/mnist_png/training/1'),Path('/Users/adgb/.fastai/data/mnist_png/training/8'),Path('/Users/adgb/.fastai/data/mnist_png/training/4'),Path('/Users/adgb/.fastai/data/mnist_png/training/3'),Path('/Users/adgb/.fastai/data/mnist_png/training/2'),Path('/Users/adgb/.fastai/data/mnist_png/training/5')]

In [7]:
# Review testing directory
(path/'testing').ls()

(#10) [Path('/Users/adgb/.fastai/data/mnist_png/testing/9'),Path('/Users/adgb/.fastai/data/mnist_png/testing/0'),Path('/Users/adgb/.fastai/data/mnist_png/testing/7'),Path('/Users/adgb/.fastai/data/mnist_png/testing/6'),Path('/Users/adgb/.fastai/data/mnist_png/testing/1'),Path('/Users/adgb/.fastai/data/mnist_png/testing/8'),Path('/Users/adgb/.fastai/data/mnist_png/testing/4'),Path('/Users/adgb/.fastai/data/mnist_png/testing/3'),Path('/Users/adgb/.fastai/data/mnist_png/testing/2'),Path('/Users/adgb/.fastai/data/mnist_png/testing/5')]

# Fastai classifier

In [10]:
# Build data loader
dls = ImageDataLoaders.from_folder(path,train='training' ,valid='testing')

In [12]:
learn = vision_learner(dls,resnet18,pretrained=False,
                       loss_func=F.cross_entropy,metrics=accuracy)

In [13]:
learn.fit_one_cycle(2,0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.682662,6.052979,0.955500,04:09
1,0.078797,0.035033,0.990000,04:11


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati